In [12]:
# REQUIREMENTS
#   - see https://www.mosek.com/products/academic-licenses/ to get a .lic file and place it in the required directory
#   - then "pip install mosek"

import os
if os.path.basename(os.getcwd()) != 'Learning-Dynamic-Systems': os.chdir(".\\..") 

import numpy as np
from optimal_controller.state_space import generate_perturbed
from optimal_controller.optimal_controls import *
import logging
logging.basicConfig(level=logging.INFO, force=True)

In [13]:
# n_states = 1
# n_inputs = 1
# A0 = np.array([[0.7]])
# B0 = np.array([[0.4]])
# n_plants = 2000
# A_list = generate_perturbed(A0, n_plants, 0.1)
# B_list = np.ones_like(A_list) * 0.4

In [14]:
n_states = 2
n_inputs = 2
A0 = np.array([[1, 0.5],
               [0.5, 1]])
B0 = np.eye(2)
n_plants = 128
A_list = generate_perturbed(A0, n_plants, 0.1)
B_list = generate_perturbed(B0, n_plants, 0.1)

In [15]:
# n_states = 4
# n_inputs = 1
# A0 = np.array([[0, 0, 0, -0.048],
#                [1, 0, 0, 0.676],
#                [0, 1, 0, -2.21],
#                [0, 0, 1, 2.6]])
# B0 = np.array([[-0.12], [0.68], [-1.4], [1]])
# n_plants = 128
# A_list = generate_perturbed(A0, n_plants, 0.005)
# B_list = generate_perturbed(B0, n_plants, 0.005)

In [16]:
# n_states = 6
# n_inputs = 1
# A0 = np.array([[0,  0,  0,  0,  0, -0.085],
#                [1,  0,  0,  0,  0,  0.642],
#                [0,  1,  0,  0,  0, -2.219],
#                [0,  0,  1,  0,  0,  4.354],
#                [0,  0,  0,  1,  0, -5.11 ],
#                [0,  0,  0,  0,  1,  3.4  ]])
# B0 = np.array([[-0.0068], [0.0796], [-0.4170], [1.17], [-1.7], [1]])
# n_plants = 256
# A_list = generate_perturbed(A0, n_plants, 0.005)
# B_list = generate_perturbed(B0, n_plants, 0.002)

In [17]:
K = get_optimal_controller(A_list, B_list)

INFO:optimal_controller.optimal_controls: Defined 22 LMI constraints


INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[0.990 0.443]
 [0.502 0.897]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.847 0.707]
 [0.513 0.867]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 6.3455
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 5.8530


In [18]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'all_plants'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Defined 128 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[0.999 0.451]
 [0.521 0.940]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.839 0.708]
 [0.512 0.865]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 6.3099
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 5.8530


In [19]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'convhull_oliveira'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Convex hull of A set has 10 vertices
INFO:optimal_controller.optimal_controls: Convex hull of B set has 2 vertices
INFO:optimal_controller.optimal_controls: This will lead to 20 LMI constraints
INFO:optimal_controller.optimal_controls: Defined 20 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[1.031 0.572]
 [0.440 0.920]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.845 0.708]
 [0.517 0.868]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 6.0789
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 5.8530


In [20]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'convhull_peaucelle'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'vertex_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Defined 22 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[0.990 0.443]
 [0.502 0.897]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.847 0.707]
 [0.513 0.867]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 6.3455
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 5.8530


In [21]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'all_plants'
ocSolver.simstab_method = 'riccati'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Simultaneously stabilising feedback gain is K = [[0.853 0.720]
 [0.509 0.819]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.850 0.708]
 [0.501 0.866]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 6.0327
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 5.8529


In [22]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'lowresMVEE'
ocSolver.set_max_n_verts(1000)
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controller.optimal_controls: Defined 114 LMI constraints
INFO:optimal_controller.optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[1.007 0.449]
 [0.497 0.976]]
INFO:optimal_controller.optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[0.847 0.707]
 [0.512 0.867]]
INFO:optimal_controller.optimal_controls: Worst J using initial K: 6.3124
INFO:optimal_controller.optimal_controls: Worst J using optimized K: 5.8529
